In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/MGU-CGANMusicConverter

/content/drive/MyDrive/Colab Notebooks/MGU-CGANMusicConverter


In [3]:
import os
import pickle
import numpy as np
from glob import glob
from keras.utils.np_utils import to_categorical

In [4]:
def get_data(file_list):
  def load_into(filename, x, y):
    with open(filename, 'rb') as f:
      pkl_file = pickle.load(f)
      x.append(pkl_file['audio'])
      y.append(int(pkl_file['class_id']))

  x, y = [], []
  for filename in file_list:
    load_into(filename, x, y)

  return np.array(x), np.array(y)

In [5]:
OUTPUT_DIR_TRAIN = './audio_splitting/output/train'
OUTPUT_DIR_TEST = './audio_splitting/output/test'
num_classes = 10

train_files = glob(os.path.join(OUTPUT_DIR_TRAIN, '**.pkl'))
x_train, y_train = get_data(train_files)

y_train = to_categorical(y_train, num_classes=num_classes)

test_files = glob(os.path.join(OUTPUT_DIR_TEST, '**.pkl'))
x_test, y_test = get_data(test_files)
y_test = to_categorical(y_test, num_classes=num_classes)

print('x_train.shape =', x_train.shape)
print('y_train.shape =', y_train.shape)
print('x_test.shape =', x_test.shape)
print('y_test.shape =', y_test.shape)
print('x_train.len =', len(x_train))
print('y_train.len =', len(y_train))
print('x_test.len =', len(x_test))
print('y_test.len =', len(y_test))
print(y_train[4799])

x_train.shape = (4800, 110250, 1)
y_train.shape = (4800, 10)
x_test.shape = (1200, 110250, 1)
y_test.shape = (1200, 10)
x_train.len = 4800
y_train.len = 4800
x_test.len = 1200
y_test.len = 1200
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
from keras.backend import in_top_k
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from keras.layers.core import Activation
from keras.layers import LeakyReLU

AUDIO_LENGTH = 110250

def add_residual_block(model):
  model.add(Conv1D(
      128,
      kernel_size=3,
      strides=1,
      #padding='same',
      #kernel_regularizer=regularizers.l2(l=0.0001)      
  ))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Conv1D(
      128,
      kernel_size=3,
      strides=1,
      #padding='same',
      #kernel_regularizer=regularizers.l2(l=0.0001)      
  ))
  model.add(BatchNormalization())

  model.add(LeakyReLU())

def classifier(num_classes=10):
  m = Sequential()
  m.add(Conv1D(
      128,
      input_shape=[AUDIO_LENGTH, 1],
      kernel_size=3,
      strides=3,
      #padding='same',
      #kernel_regularizer=regularizers.l2(l=0.0001)
  ))
  add_residual_block(m)